Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fneon-city-overdrive-171498%2Fneon_city_overdrive.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/neon-city-overdrive-171498)]*

## [``dyce``](https://posita.github.io/dyce/) translation and comparison of [Carcer’s](https://rpg.stackexchange.com/a/171505/71245) and [Karonen’s](https://rpg.stackexchange.com/a/194712/71245) solutions to “[How to calculate the probabilities for eliminative dice pools (dice cancelling mechanic) in Neon City Overdrive?](https://rpg.stackexchange.com/a/195045/71245)”

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

try:
    import neon_city_overdrive
except ImportError:
    # Work-around for JupyterLite in non-Chromium browsers
    import js
    import os
    from urllib.parse import urljoin, urlparse, urlunparse
    loc_url = urlparse(js.location.toString())
    ext_root = loc_url.path.find("/extensions/@jupyterlite/")
    if ext_root < 0:
        base_url = urljoin(js.location.toString(), "../files/")
    else:
        loc_url = loc_url._replace(path=loc_url.path[:ext_root])
        base_url = urljoin(urlunparse(loc_url), "files/")
    for path in (
                "stack-exchange/neon-city-overdrive-171498/neon_city_overdrive.py",
            ):
        url = urljoin(base_url, path)
        res = await js.fetch(url)
        assert 200 <= res.status < 300
        text = await res.text()
        with open(os.path.basename(path), "w") as f:
            f.write(text)
    import neon_city_overdrive

Substantive code is in [``neon_city_overdrive.py``](neon_city_overdrive.py).

In [2]:
import dyce  # pre-import dyce to make sure it's excluded from our time measurements below

def do_it(nco_func, a_range, d_range):
    return [
        (f"{a_pool_size}d6 vs. {d_pool_size}d6", nco_func(a_pool_size, d_pool_size))
        for a_pool_size in a_range
        for d_pool_size in d_range
    ]

In [3]:
%%timeit  # Carcer - [1-4]d6 vs. [0-4]d6
%run neon_city_overdrive.py
do_it(nco_carcer, range(1, 5), range(5))

304 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit  # Karonen - [1-4]d6 vs. [0-4]d6
%run neon_city_overdrive.py  # (re)defines nco function for getting accurate first-run (pre-memoization) measurements
do_it(nco_karonen, range(1, 5), range(5))

20.9 ms ± 469 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit  # Karonen - [1-11]d6 vs. [0-11]d6
%run neon_city_overdrive.py
do_it(nco_karonen, range(1, 12), range(12))  # these dice go to eleven!

421 ms ± 7.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# Check that each produces equivalent results
%run neon_city_overdrive.py

for a_pool_size in range(1, 4):
    for d_pool_size in range(4):
        assert (
            nco_carcer(a_pool_size, d_pool_size)
            == nco_karonen(a_pool_size, d_pool_size)
        ), (
            f"nco_carcer({a_pool_size}, {d_pool_size})"
            f" != nco_karonen({a_pool_size}, {d_pool_size})"
        )

In [7]:
from anydyce import jupyter_visualize
from neon_city_overdrive import nco_karonen

a_range = tuple(range(1, 6))
d_range = tuple(range(6))

# Visualize [1-5]d6 vs. [0-5]d6 with burst graphs
jupyter_visualize(do_it(nco_karonen, a_range, d_range))

## [``dyce``](https://posita.github.io/dyce/) solution to [“Anydice: Neon City Overdrive type eliminative dice pool BUT danger dice cancel equal to AND less than”](https://rpg.stackexchange.com/a/198632/71245) and comparison

I recently [added the ``nco_so_dangerous`` implementation to ``neon_city_overdrive.py``](https://github.com/posita/dyce-notebooks/blob/main/notebooks/stack-exchange/neon-city-overdrive-171498/neon_city_overdrive.py#L132-L177). Below is a comparison of the two mechanics. Gray outer rings show the basic mechanic. Red inner rings show the dangerous cancelation modification.

In [8]:
from neon_city_overdrive import nco_so_dangerous

jupyter_visualize(
    (
        (f"{a}d6 vs. {d}d6", nco_so_dangerous(a, d), nco_karonen(a, d))
        for a in a_range for d in d_range
    ),
    initial_burst_cmap_inner="Reds",
    initial_burst_cmap_outer="gist_gray_r",
    initial_burst_cmap_link=False,
)